In [1]:
import os

from IPython import display
# import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random as python_random

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import regularizers

from lazypredict.Supervised import LazyClassifier

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
yamnet_model = hub.load(yamnet_model_handle) # if there's an "SavedModel file does not exist at:", delete that folder and rerun it


2023-03-03 10:29:11.409306: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-03 10:29:19.020892: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df_all = pd.read_csv('../train_val_test_split/train_val_test_GoogleAudioSet.csv', index_col=0)
df_all

,file,source,category,weight,fold
0,../data/interim/GoogleAudioSet_unbalanced_list...,Google_city,1,1,0
1,../data/interim/GoogleAudioSet_unbalanced_list...,Google_city,1,1,8
2,../data/interim/GoogleAudioSet_unbalanced_list...,Google_nature,0,1,5
3,../data/interim/GoogleAudioSet_unbalanced_list...,Google_city,1,1,1
4,../data/interim/GoogleAudioSet_unbalanced_list...,Google_city,1,1,1
...,...,...,...,...,...
13662,../data/interim/GoogleAudioSet_unbalanced_list...,Google_city,1,1,5
13663,../data/interim/GoogleAudioSet_unbalanced_list...,Google_city,1,1,3
13664,../data/interim/GoogleAudioSet_unbalanced_list...,Google_city,1,1,8
13665,../data/interim/GoogleAudioSet_unbalanced_list...,Google_city,1,1,8


In [3]:
# check the weight
df_all.groupby(['source'])['weight'].sum()

source
Google_city      6687
Google_nature    6980
Name: weight, dtype: int64

In [4]:
def extract_yamnet_embedding(filename):
    import pickle
    
    file = open(filename, 'rb')
    output = pickle.load(file)
#     output = pd.read_pickle(file)
    wav_raw = output['y']
    wav_bg = output['bg_y']
    wav_fg = output['fg_y']
    file.close()

# # no need to run padding as the time dimension will be collapsed after transforming to embeddings anyway    
#     if len(wav_raw) < 160000:
#         wav_raw = np.pad(wav_raw, (0,160000-len(wav_raw)),'mean')

#     if len(wav_bg) < 160000:
#         wav_bg = np.pad(wav_bg, (0,160000-len(wav_bg)),'mean')

#     if len(wav_fg) < 160000:
#         wav_fg = np.pad(wav_fg, (0,160000-len(wav_fg)),'mean')

    # Extract YAMNet embeddings for each frame
    scores, embedding_tensor_raw, spectrogram = yamnet_model(wav_raw)
    embedding_tensor_raw = tf.reduce_mean(embedding_tensor_raw, axis=0).numpy()

    scores, embedding_tensor_bg, spectrogram = yamnet_model(wav_bg)
    embedding_tensor_bg = tf.reduce_mean(embedding_tensor_bg, axis=0).numpy()

    scores, embedding_tensor_fg, spectrogram = yamnet_model(wav_fg)
    embedding_tensor_fg = tf.reduce_mean(embedding_tensor_fg, axis=0).numpy()
        
    
    return embedding_tensor_raw, embedding_tensor_bg, embedding_tensor_fg

In [5]:
from joblib import Parallel, delayed

embeddings_raw_list = []
embeddings_bg_list = []
embeddings_fg_list = []
import time
start_time = time.time()


for index, row in df_all.iterrows():
    embedding_tensor_raw, embedding_tensor_bg, embedding_tensor_fg = extract_yamnet_embedding(row['file'])
    
    embeddings_raw_list.append(embedding_tensor_raw)
    embeddings_bg_list.append(embedding_tensor_bg)
    embeddings_fg_list.append(embedding_tensor_fg)
    
embeddings_raw_matrix = np.stack(embeddings_raw_list, axis=0)
embeddings_bg_matrix = np.stack(embeddings_bg_list, axis=0)
embeddings_fg_matrix = np.stack(embeddings_fg_list, axis=0)
    
print('seconds: '+str(time.time()-start_time))


seconds: 2416.0300312042236


In [6]:
embeddings_matrix = np.concatenate((embeddings_raw_matrix, embeddings_bg_matrix, embeddings_fg_matrix), axis=1)

In [7]:
train_index = df_all['fold']<8
valid_index = df_all['fold']==8
test_index = df_all['fold']==9

# Raw signal

In [8]:
X_train = embeddings_raw_matrix[train_index,:]
y_train = df_all.loc[train_index,'category']
X_valid = embeddings_raw_matrix[valid_index,:]
y_valid = df_all.loc[valid_index,'category']

transformer = Pipeline(steps=[
       ('imputer', SimpleImputer(missing_values=np.nan, strategy='mean'))
      ,('scaler', StandardScaler())
])

transformer.fit(X_train)
X_train = transformer.transform(X_train)
X_valid = transformer.transform(X_valid)


clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_valid, y_train, y_valid)
models

100%|███████████████████████████████████████████| 29/29 [07:17<00:00, 15.10s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.70,0.70,0.70,0.70,4.05
SVC,0.70,0.70,0.70,0.70,59.17
RandomForestClassifier,0.70,0.70,0.70,0.70,14.03
ExtraTreesClassifier,0.69,0.69,0.69,0.69,6.07
RidgeClassifier,0.69,0.69,0.69,0.69,1.20
AdaBoostClassifier,0.69,0.69,0.69,0.69,32.65
XGBClassifier,0.69,0.69,0.69,0.69,24.07
LinearDiscriminantAnalysis,0.69,0.69,0.69,0.69,2.45
RidgeClassifierCV,0.69,0.69,0.69,0.69,1.97


# Background signal

In [9]:
X_train = embeddings_bg_matrix[train_index,:]
y_train = df_all.loc[train_index,'category']
X_valid = embeddings_bg_matrix[valid_index,:]
y_valid = df_all.loc[valid_index,'category']

transformer = Pipeline(steps=[
       ('imputer', SimpleImputer(missing_values=np.nan, strategy='mean'))
      ,('scaler', StandardScaler())
])

transformer.fit(X_train)
X_train = transformer.transform(X_train)
X_valid = transformer.transform(X_valid)


clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_valid, y_train, y_valid)
models

100%|███████████████████████████████████████████| 29/29 [07:32<00:00, 15.60s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
RidgeClassifierCV,0.71,0.71,0.71,0.71,1.87
RidgeClassifier,0.70,0.70,0.70,0.70,1.11
LinearDiscriminantAnalysis,0.70,0.70,0.70,0.70,2.52
SVC,0.70,0.70,0.70,0.70,60.89
LogisticRegression,0.70,0.70,0.70,0.70,1.07
AdaBoostClassifier,0.69,0.69,0.69,0.69,32.05
CalibratedClassifierCV,0.69,0.69,0.69,0.69,107.32
LGBMClassifier,0.69,0.69,0.69,0.69,3.97
RandomForestClassifier,0.69,0.69,0.69,0.69,13.30


# Foreground signal

In [10]:
X_train = embeddings_fg_matrix[train_index,:]
y_train = df_all.loc[train_index,'category']
X_valid = embeddings_fg_matrix[valid_index,:]
y_valid = df_all.loc[valid_index,'category']

transformer = Pipeline(steps=[
       ('imputer', SimpleImputer(missing_values=np.nan, strategy='mean'))
      ,('scaler', StandardScaler())
])

transformer.fit(X_train)
X_train = transformer.transform(X_train)
X_valid = transformer.transform(X_valid)


clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_valid, y_train, y_valid)
models

100%|███████████████████████████████████████████| 29/29 [09:19<00:00, 19.30s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
SVC,0.66,0.66,0.66,0.66,66.57
LogisticRegression,0.66,0.66,0.66,0.66,1.07
LGBMClassifier,0.65,0.65,0.65,0.65,4.30
AdaBoostClassifier,0.65,0.65,0.65,0.65,48.74
RandomForestClassifier,0.65,0.65,0.65,0.65,21.11
RidgeClassifierCV,0.65,0.65,0.65,0.65,2.34
LinearDiscriminantAnalysis,0.65,0.65,0.65,0.65,2.39
ExtraTreesClassifier,0.65,0.65,0.65,0.65,6.23
RidgeClassifier,0.65,0.65,0.65,0.65,0.41


# All signal

In [11]:
X_train = embeddings_matrix[train_index,:]
y_train = df_all.loc[train_index,'category']
X_valid = embeddings_matrix[valid_index,:]
y_valid = df_all.loc[valid_index,'category']

transformer = Pipeline(steps=[
       ('imputer', SimpleImputer(missing_values=np.nan, strategy='mean'))
      ,('scaler', StandardScaler())
])

transformer.fit(X_train)
X_train = transformer.transform(X_train)
X_valid = transformer.transform(X_valid)


clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_valid, y_train, y_valid)
models


100%|███████████████████████████████████████████| 29/29 [25:16<00:00, 52.30s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
SVC,0.70,0.70,0.70,0.70,188.48
LogisticRegression,0.69,0.69,0.69,0.69,2.80
NuSVC,0.69,0.69,0.69,0.69,350.60
LGBMClassifier,0.69,0.69,0.69,0.69,12.60
RandomForestClassifier,0.69,0.69,0.69,0.69,30.07
RidgeClassifierCV,0.69,0.69,0.69,0.69,17.61
AdaBoostClassifier,0.69,0.69,0.69,0.69,116.13
ExtraTreesClassifier,0.68,0.68,0.68,0.68,12.30
XGBClassifier,0.68,0.68,0.68,0.68,83.56


In [12]:
  
    
    
    

# # Define a function to extract YAMNet embeddings for a single audio file
# def extract_yamnet_embedding(audio_path):
#     # Load the audio file
#     audio, sample_rate = tf.audio.decode_wav(tf.io.read_file(audio_path))
    
#     # Extract YAMNet embeddings for each frame
#     embedding_tensor = yamnet_model(audio)
#     embeddings = tf.reduce_mean(embedding_tensor, axis=0)
    
#     return embeddings

# # Define a function to parse a single metadata file
# def parse_metadata(metadata_path):
#     # Implement your metadata parsing code here
#     metadata = ...
    
#     return metadata

# # Define a function to combine YAMNet embeddings and metadata for a single audio file
# def combine_features(audio_path, metadata_path):
#     # Extract YAMNet embeddings
#     yamnet_embedding = extract_yamnet_embedding(audio_path)
    
#     # Parse metadata
#     metadata = parse_metadata(metadata_path)
    
#     # Concatenate the YAMNet embedding and metadata into a single feature vector
#     feature_vector = tf.concat([yamnet_embedding, metadata], axis=-1)
    
#     return feature_vector

# # Define a function to load and preprocess a single data sample
# def load_and_preprocess_data(audio_path, metadata_path):
#     # Combine YAMNet embeddings and metadata
#     feature_vector = combine_features(audio_path, metadata_path)
    
#     # Implement your data preprocessing code here
#     preprocessed_data = ...
    
#     return preprocessed_data

# # Load a list of audio and metadata file paths
# audio_paths = ["path/to/audio1.wav", "path/to/audio2.wav", ...]
# metadata_paths = ["path/to/metadata1.csv", "path/to/metadata2.csv", ...]

# # Create a TensorFlow dataset using the audio and metadata file paths
# dataset = tf.data.Dataset.from_tensor_slices((audio_paths, metadata_paths))

# # Use the map function to apply the load_and_preprocess_data function to each sample in the dataset
# dataset = dataset.map(load_and_preprocess_data)

# # Implement your training code here using the preprocessed dataset
# ...